# QC a Processed Collection

In [ ]:
%load_ext autoreload
%autoreload 2

from src import *
from src.setup import *
from src.process import *
from src.qc import *

### `parameters`

In [ ]:
collection_name = "" # Define Collection name - example: "ble_12100302_Medina"
# collection_name = "Baxter_0_7_0"

### Instantiate CollectionData, Database, and JobClient objects

In [ ]:
collection = CollectionData(collection_name)
database = Database(collection)
job_client = JobClient(collection)

### Create Excel Error Report

In [ ]:
%%time
dfs = []
for process_name in ["conflate_model"]:
    # Capture the final status of unknown status jobs
    job_client.poll_and_update_job_status(database, process_name, "models")
    _, _, failed_reaches = database.get_reach_status_by_process(process_name, "models")
    df = job_client.get_failed_jobs_df(failed_reaches)
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, process_name, collection.error_report_path)

In [ ]:

%%time
dfs = []
for process_name in [
    "extract_submodel",
    "create_ras_terrain",
    "create_model_run_normal_depth",
    "run_incremental_normal_depth",
    "run_iknown_wse",
    "create_irating_curves_db",
    "run_known_wse",
    "create_rating_curves_db",
    "create_fim_lib",
]:
    # Capture the final status of unknown status jobs
    job_client.poll_and_update_job_status(database, process_name)
    _, _, failed_reaches = database.get_reach_status_by_process(process_name)
    df = job_client.get_failed_jobs_df(failed_reaches)
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, process_name, collection.error_report_path)

### Create QC Map

In [ ]:
copy_qc_map(collection)

### Create Composite Rasters (flows2fim)

In [ ]:
%%time

run_flows2fim(collection)